In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt # for plotting
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
# load the data
img_to_tensor = transforms.ToTensor()
mnist_data = datasets.MNIST('data', train=True, download=True, transform=img_to_tensor)
mnist_data = list(mnist_data)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
base_dir = '/content/gdrive/MyDrive/aps360/project/test/imagesaving/'

In [ ]:
def generateMatrices(data, rows, cols, num=0, directory='', valid=True):
  if num < 1:
    num = int(len(data) / (rows * cols))
  inds = list(range(len(data)))
  np.random.shuffle(inds)

  i = 0
  for matrix in range(num):
    baseimg = data[inds[i]][0]
    if not valid:
      blanksquare = torch.zeros(baseimg.shape)
      blankinds = np.random.randint(rows * cols, size=min(rows, cols)-1)
      #print(blankinds)
      if 0 in blankinds:
        baseimg = blanksquare

    for r in range(rows):
      for c in range(cols):
        if c == 0:
          if not valid and r * cols + c in blankinds:
            rowbase = blanksquare
          else:
            rowbase = data[inds[i]][0]
        else:
          i += 1
          if i >= len(data):
            i = 0
            np.random.shuffle(inds)
          if not valid and r * cols + c in blankinds:
            rowbase = torch.concat((rowbase, blanksquare), dim=2)
          else:
            rowbase = torch.concat((rowbase, data[inds[i]][0]), dim=2)
      i += 1
      if i >= len(data):
        i = 0
        np.random.shuffle(inds)
      if r == 0:
        baseimg = rowbase
      else:
        baseimg = torch.concat((baseimg, rowbase), dim=1)
    baseimg = transforms.functional.invert(baseimg)
    #plt.imshow(baseimg.squeeze(), cmap='Greys')
    #plt.show()
    if directory != '':
      torchvision.utils.save_image(baseimg, directory + "matrix_{}_{}x{}_{}.jpg".format(
          'valid' if valid else 'invalid', rows, cols, matrix
      ))


In [ ]:
np.random.seed(1337)
generateMatrices(mnist_data, 3, 3, num=6000, directory=(base_dir + '3x3/'))
generateMatrices(mnist_data, 3, 3, num=500, directory=(base_dir + 'fake3x3/'), valid=False)